In [25]:
# MSM 107 analysis
# ROSINA data
# plotting individual particle profiles

import pandas as pd
import numpy as np
import os, glob, statistics, math, gsw
from tkinter import filedialog
import matplotlib.pyplot as plt

In [26]:
particle_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/03_ROSINA/_Results/08_ResultsPython/08_ResultsPython")
particle_station_list = [
"Rosi01/IdroCTD-02_Img1330removed/BinnedCTD",
"Rosi03/IdroCTD-02/BinnedCTD", "Rosi04/IdroCTD-02/BinnedCTD",
"Rosi05/IdroCTD-02/BinnedCTD", "Rosi06/IdroCTD-02/BinnedCTD",
"Rosi07/IdroCTD-02_Img148Removed/BinnedCTD","Rosi08/IdroCTD-02-Img119removed/BinnedCTD",
"Rosi10/IdroCTD-02/BinnedCTD", "Rosi11/IdroCTD-02/BinnedCTD",
"Rosi12/IdroCTD-02/BinnedCTD","Rosi14/IdroCTD-02/BinnedCTD",
"Rosi15/IdroCTD-02/BinnedCTD","Rosi16/IdroCTD-02/BinnedCTD",
"Rosi17/IdroCTD-02/BinnedCTD","Rosi18/IdroCTD-02/BinnedCTD",
"Rosi19/IdroCTD-02/BinnedCTD","Rosi20/IdroCTD-02/BinnedCTD"
]
#"Rosi02/ManDepth-Img584u577removed/BinnedCTD"
CTD_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/04_CTD-Idronaut")
CTD_station_list = [
"01-Profile01/MSM107-P001-C02.txt", "02-Profile03/MSM107-P003-C01.txt",
"03-Profile04/MSM107-P004-C01.txt", "04-Profile05/MSM107-P005-C01.txt",
"05-Profile06/MSM107-P006-C01.txt", "06-Profile07/MSM107-P007-C03.txt",
"07-Profile08/MSM107-P008-C01.txt", "08-Profile10/MSM107-P010-C01.txt",
"09-Profile11/MSM107-P011-C01.txt", "10-Profile12/MSM107-P012-C03.txt",
"12-Profile14/MSM107-P014-C01.txt", "13-Profile15/MSM107-P015-C01.txt",
"14-Profile16/MSM107-P016-C01.txt", "15-Profile17/MSM107-P017-C02.txt",
"16-Profile18/MSM107-P018-C01.txt", "17-Profile19/MSM107-P019-C01.txt",
"18-Profile20/MSM107-P020-C02.TXT"
]

plot_path = os.path.abspath("/Volumes/SEGATE1T/MSM107/plot")

In [27]:
# some definitions
# binned definition
def binned (values, depths, bin):
    df = pd.DataFrame(list(zip(values, depths)), columns=["value", "depth"])
    max_d = max(depths)
    nBins = int(max_d / bin) + 1 # 5 m bins
    bin_value = []
    bin_depth = []
    for n in range(0, nBins):
        each_df = df.loc[(df["depth"] > n*bin) & (df["depth"] < (n+1)*bin) ]
        if each_df.empty == True:
            bin_value.append(np.nan)
            bin_depth.append(n*5)
        else:
            value_mean = each_df["value"].mean()
            bin_value.append(value_mean) 
            bin_depth.append(n*5)
    return bin_value, bin_depth

In [28]:
for k in range(0, len(particle_station_list)):
    p = particle_station_list[k]
    c = CTD_station_list[k]

    # reading in data
    ##########################
    # reading in particle data
    filePath = os.path.expanduser( os.path.join(particle_path, p) )
    pathParticlesBinned = filePath + "/ParticlesBinned.npy"
    pathVolumeBinned = filePath + "/VolumeBinned.npy"
    #pathAreaBinned = filePath + "/AreaBinned.npy"
    pathDepth = filePath + "/Depth.npy"
    pathConfigFile = filePath +"/ConfigFileHeader.txt"

    # create header and list for dataframe
    particlesBinnedList = []          # list for input data numpy files
    volumeBinnedList = []             # list for input data numpy files
    #areaBinnedList = []               # list for input data numpy files
    depthList = []                    # list for input data numpy files
    eSDList = []                      # list for input data numpy files

    # bins value in mm
    Bins = [0.04875, 0.061425, 0.0773955, 0.0975183, 0.122873, 0.15482, 0.195073, 0.245792, 0.309698, 
        0.39022, 0.491677, 0.619513, 0.780587, 0.983539, 1.23926, 1.56147, 1.96745, 2.47898, 3.12352, 3.93564]
    particlesBinned = np.load(pathParticlesBinned)
    volumeBinned = np.load(pathVolumeBinned)
    #areaBinned = np.load(pathAreaBinned)
    depth = np.load(pathDepth)
    #    eSD = np.load(pathESD)

    configDataFile = open(pathConfigFile, "rt")
    configDataList = configDataFile.read()
    configDataList = configDataList.split("\n")
    configDataFile.close()

    # data converting to dataframe
    numberdf = pd.DataFrame(particlesBinned, columns = Bins)
    volumedf =  pd.DataFrame(volumeBinned, columns = Bins)
    #areadf = pd.DataFrame(areaBinned, columns = Bins)
    

    # check particle size spectra vs ESD
    sum = numberdf.sum(axis = 0).to_frame()
    sumdf = pd.DataFrame(sum.T, columns = Bins)
    size_interval = [(math.floor(i * 10000)) / 10000.0 for i in Bins] # round down the bins
    size_spectra_l =[]
    esd_l = []
    for i in range(0, len(size_interval)):
        if i == len(size_interval)-1:
            break
        else:
            size_s, size_l = size_interval[i], size_interval[i+1]
            esd_l.append(statistics.mean([size_s, size_l]))
            size_sum = sumdf.loc[:, sumdf.columns[(sumdf.columns > size_s) & (sumdf.columns < size_l)]].sum(axis = 1)
            size_spectra_l.append(size_sum.sum(axis=0)/(size_l-size_s))


    # create vertical profile of particle
    numsum = numberdf.sum(axis = 1).tolist()
    volsum = volumedf.sum(axis = 1).tolist()
    #areasum = areadf.sum(axis = 1).tolist()
    meansum = []
    mediansum = []

    # calculate mean and median for each bins
    for index, row in numberdf.iterrows():
        if statistics.mean(row.values) == 0:
            meansum.append(np.nan)
            mediansum.append(np.nan)
        else:

            mean = statistics.mean( np.repeat(Bins, [int(i) for i in row.values]) )
            median = statistics.median( np.repeat(Bins, [int(i) for i in row.values]) )
            meansum.append(mean)
            mediansum.append(median)

    ###########################
    # readidng in CTD data
    CTDPath = os.path.expanduser( os.path.join(CTD_path, c) )
    CTDdf = pd.read_csv(CTDPath, sep="\t", skiprows=16, header=0, engine="python")

    ###########################
    # figures
    fig, ax = plt.subplots(nrows = 2, ncols= 4, figsize=(14, 10), facecolor="white")

    ax0 = ax[0, 0]
    ax0.loglog(esd_l, size_spectra_l)
    ax0.set_xlabel("ESD mm")
    ax0.set_ylabel("size spectra # $cm^{-4}$",size= 12)


    ax5 = ax[0, 1]
    tempbinned, ctddepthbinned = binned (CTDdf["Temperature"], CTDdf["Depth"], 5)
    ax5.plot(tempbinned, ctddepthbinned,  color="red")
    ax5.invert_yaxis()
    ax5.set_xlabel("Temperature", color = "red", size= 12)

    sec_ax5 = ax5.twiny()
    salbinned, ctddepthbinned = binned (CTDdf["Salinity"], CTDdf["Depth"], 5)
    sec_ax5.plot(salbinned, ctddepthbinned,  color="black")
    sec_ax5.set_xlabel("Salinity", color = "black", size= 12)

    ax6 = ax[0, 2]
    o2binned, ctddepthbinned = binned (CTDdf["Optical O2%"], CTDdf["Depth"], 5)
    ax6.plot(o2binned, ctddepthbinned,  color="blue")
    ax6.invert_yaxis()
    ax6.set_xlabel("Oxygen", color = "blue", size= 12)

    sec_ax6 = ax6.twiny()
    flubinned, ctddepthbinned = binned (CTDdf["Fluorometer AutoScale"], CTDdf["Depth"], 5)
    sec_ax6.plot(flubinned, ctddepthbinned,  color="green")
    sec_ax6.set_xlabel("Fluorescence", color = "green", size= 12)

    # plot TS diagram
    ax7 = ax[0, 3]

    ## generate contour for density
    mint= 0 #min(tempbinned) # contour for density
    maxt= 12 #max(tempbinned)
    mins= 35 #min(salbinned)
    maxs= 36 #max(salbinned)

    tempL=np.linspace(mint-1,maxt+1,156)
    salL=np.linspace(mins-1,maxs+1,156)

    Tg, Sg = np.meshgrid(tempL,salL)
    sigma_theta = gsw.sigma0(Sg, Tg)
    cnt = np.linspace(sigma_theta.min(), sigma_theta.max(),156) # density contour data generated

    cs = ax7.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1)
    cl=ax7.clabel(cs, fontsize=12, inline=True, fmt='%.1f')
    ###
    sc = ax7.scatter(salbinned, tempbinned, c= ctddepthbinned, s=1)
    cbar = plt.colorbar(sc, ax=ax7)
    cbar.ax.invert_yaxis()
    ax7.set_title("TS diagram", size= 12)
    ax7.set_xlabel("Salinity", size= 12)
    ax7.set_ylabel("Temperature", size= 12)
    ax7.set_ylim(1, 12.5)
    ax7.set_xlim(34.5, 36)

    # plot total number
    ax1 = ax[1, 0]
    numbinned, depthbinned = binned (numsum, depth, 5)
    ax1.plot(numbinned, depthbinned,  color="black")
    ax1.invert_yaxis()
    ax1.set_xlim(0, 200)
    ax1.set_title("total number", size= 12)
    ax1.set_xlabel("#/L", size= 12)

    # plot total volume
    ax2 = ax[1, 1]
    volbinned, depthbinned = binned (volsum, depth, 5)
    ax2.plot(volbinned, depthbinned,  color="black")
    ax2.invert_yaxis()
    ax2.set_xlim(0, 0.5)
    ax2.set_title("total volume", size= 12)
    ax2.set_xlabel("cm3/L", size= 12)

    # plot median ESD
    ax4 = ax[1, 2]
    medianbinned, depthbinned = binned (mediansum, depth, 5)
    ax4.plot(medianbinned, depthbinned,  color="black")
    ax4.invert_yaxis()
    ax4.set_xlim(0, 0.4)
    ax4.set_title("ESD median", size= 12)
    ax4.set_xlabel("ESD mm", size= 12)

    # plot mean ESD
    ax3 = ax[1, 3]
    meanbinned, depthbinned = binned (meansum, depth, 5)
    ax3.plot(meanbinned, depthbinned,  color="black")
    ax3.invert_yaxis()
    ax3.set_xlim(0, 0.4)
    ax3.set_title("ESD mean", size= 12)
    ax3.set_xlabel("ESD mm", size= 12)

    fig.tight_layout()

    figname = str(p).split(os.sep)[0]
    figpath = os.path.expanduser(os.path.join(plot_path, figname) )
    fig.savefig(figpath, dpi=300, facecolor="white")
    plt.close()